# Assignment 3 - Task 3

Group: David, Christian

## Approach/general idea

### 3.1

Define/describe a way to calculate or describe how diverse a number of the original explanations are as a set. Base diversity on demographics of the original dataset: gender, profession, age, ...

### 3.2

Report on maximum readability and semantic similarity of consolidated explanations. Base this on task 2 or generate explanations based on arbitrary (?) sets of original explanations to see what the max can be.

### 3.3

Disregard readability. We should know the maximum semantic similarity we achieved for 3.2.
Now we choose sets of original explanations in such a way, that the diversity is maximized, and consolidate explanations and hope they reach our max semantic similarity. Report how diverse the set can be while still being at max semantic similarity.